<a href="https://colab.research.google.com/github/mugalan/classical-mechanics-from-a-geometric-point-of-view/blob/main/mechanics/answers-to-selected-assignments/Sample_answer_to_motion_in_earth_moving_frame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Python modules

In [ ]:
import numpy as np
import scipy as sp
from scipy.integrate import odeint
import plotly.graph_objects as go

import sympy as sym
from sympy import symbols
from sympy import *

from sympy.physics.mechanics import dynamicsymbols, init_vprinting
init_vprinting()

In [ ]:
# Install the simulation helpers
!pip install --quiet "git+https://github.com/mugalan/classical-mechanics-from-a-geometric-point-of-view.git#egg=rigid-body-sim"
import sims
mr = sims.RigidBodySim()

# Sample answer to description of motion in the earth moving frame

Consider five orthonormal frames $\mathbf{e}$, $\mathbf{a}$, $\mathbf{c}$, $\mathbf{d}$ and $\mathbf{b}$. The frames $\mathbf{e}$ and $\mathbf{a}$ have coinciding origin $O_s$ and $\mathbf{e}_3=\mathbf{a}_3$ and $\mathbf{a}$ is a counter clockwise rotation of $\alpha$ about $\mathbf{e}_3$. The frame $\mathbf{c}$ is parallel to $\mathbf{a}$ with the origin $O_e$ located $r_o$ units along the $\mathbf{a}_1$ axis. The frame $\mathbf{d}$ and frame $\mathbf{c}$ have coinciding origins with $\mathbf{c}_3=\mathbf{d}_3$ and $\mathbf{d}$ is a counter clockwise rotation of $\beta$ about $\mathbf{c}_3$. The frame $\mathbf{f}$ is such that $\mathbf{d}_1=\mathbf{f}_1$ and $\mathbf{f}$ is a counter clockwise rotation of $\phi$ about $\mathbf{d}_1$. The origin of $\mathbf{f}$ and $\mathbf{d}$ coincide.


The frame $\mathbf{b}$ is such that it is palarallel to $\mathbf{f}$ and its origin $O$ is located at a distance $r_e$ along $\mathbf{f}_2$.


 and
 with coinciding origins. At any given time instant $t$ the frame $\mathbf{c}$ is related to the frame $\mathbf{e}$ by a counter clockwise rotation about the third axis by an angle $\phi=\alpha\, t$ and the frame $\mathbf{b}$ is related to the frame $\mathbf{c}$ by a counter clockwise rotation about the first axis by an angle $\theta=\beta \, t$ where $\alpha$ and $\beta$ are constant. Let $\mathbf{b}=\mathbf{e}R(t)$ where $R(t)\in SO(3)$.

A particle $P$ of mass $m$ is moving in space in such a way that it oscillates about the origin of the frames along the $\mathbf{b}_2$ axis at a frequency of $\omega$ rad/s with an amplitude of $r$ m. That is if $y$ is the displacement of $P$ along the $\mathbf{b}_2$ axis then $y(t)=r\cos(\omega t)$.

## Frame relationship

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Adjustable parameters
alpha = np.deg2rad(30)   # rotation of a about e3
beta = np.deg2rad(45)    # rotation of d about c3
phi = np.deg2rad(35)     # rotation of f about d1
r_o = 10.0                # translation of c along a1
r_e = 2.5                # translation of b along f2 (and sphere radius)

# Rotation matrices
def rot_z(theta):
    return np.array([
        [np.cos(theta), -np.sin(theta), 0],
        [np.sin(theta),  np.cos(theta), 0],
        [0,              0,             1]
    ])

def rot_x(theta):
    return np.array([
        [1, 0, 0],
        [0, np.cos(theta), -np.sin(theta)],
        [0, np.sin(theta),  np.cos(theta)]
    ])

# Base frame e
e_axes = np.eye(3)
O_s = np.array([0, 0, 0])

# Frame a (rotation about e3)
a_axes = rot_z(alpha)
# Frame c (parallel to a, translated along a1)
O_e = O_s + r_o * a_axes[:, 0]
c_axes = a_axes
# Frame d (rotation about c3)
d_axes = c_axes @ rot_z(beta)
# Frame f (rotation about d1)
f_axes = d_axes @ rot_x(phi)
# Frame b (parallel to f, translated along f2)
O = O_e + r_e * f_axes[:, 1]
b_axes = f_axes

# Colors for frames
colors = {
    "e": "black",
    "a": "red",
    "c": "blue",
    "d": "green",
    "f": "orange",
    "b": "purple"
}

# Function to create quiver-like axes for a frame
def frame_traces(origin, axes, color):
    traces = []
    for i in range(3):
        x = [origin[0], origin[0] + axes[0, i]]
        y = [origin[1], origin[1] + axes[1, i]]
        z = [origin[2], origin[2] + axes[2, i]]
        traces.append(go.Scatter3d(
            x=x, y=y, z=z,
            mode="lines",
            line=dict(color=color, width=6),
            showlegend=False
        ))
    return traces

# Assemble all frame traces
frames = []
frames += frame_traces(O_s, e_axes, colors["e"])
frames += frame_traces(O_s, a_axes, colors["a"])
frames += frame_traces(O_e, c_axes, colors["c"])
frames += frame_traces(O_e, d_axes, colors["d"])
frames += frame_traces(O_e, f_axes, colors["f"])
frames += frame_traces(O, b_axes, colors["b"])

fig = go.Figure(data=frames)

# Label origins Oₛ, Oₑ, O
for point, name in zip([O_s, O_e, O], ["Oₛ", "Oₑ", "O"]):
    fig.add_trace(go.Scatter3d(
        x=[point[0]], y=[point[1]], z=[point[2]],
        mode="text",
        text=[name],
        textfont=dict(size=18, color="black"),
        showlegend=False
    ))

# Label endpoints of b's three axes as b₁, b₂, b₃
b_axis_endpoints = [O + b_axes[:, i] for i in range(3)]
for i, point in enumerate(b_axis_endpoints, start=1):
    fig.add_trace(go.Scatter3d(
        x=[point[0]], y=[point[1]], z=[point[2]],
        mode="text",
        text=[f"b{i}"],
        textfont=dict(size=16, color=colors["b"]),
        showlegend=False
    ))

# Dashed connection lines between key origins
fig.add_trace(go.Scatter3d(
    x=[O_s[0], O_e[0]], y=[O_s[1], O_e[1]], z=[O_s[2], O_e[2]],
    mode="lines",
    line=dict(color="gray", width=4, dash="dash"),
    showlegend=False
))
fig.add_trace(go.Scatter3d(
    x=[O_e[0], O[0]], y=[O_e[1], O[1]], z=[O_e[2], O[2]],
    mode="lines",
    line=dict(color="gray", width=4, dash="dash"),
    showlegend=False
))

# === Transparent sphere centered at O_e ===
u = np.linspace(0, 2 * np.pi, 40)
v = np.linspace(0, np.pi, 20)
x_s = O_e[0] + r_e * np.outer(np.cos(u), np.sin(v))
y_s = O_e[1] + r_e * np.outer(np.sin(u), np.sin(v))
z_s = O_e[2] + r_e * np.outer(np.ones_like(u), np.cos(v))

fig.add_trace(go.Surface(
    x=x_s, y=y_s, z=z_s,
    opacity=0.15,
    showscale=False,
    colorscale=[[0, 'lightblue'], [1, 'lightblue']],
    hoverinfo='skip'
))

# === Full circle in e1-e2 plane centered at O_s, radius r_o ===
theta_circle = np.linspace(0, 2 * np.pi, 200)
x_circ = O_s[0] + r_o * np.cos(theta_circle)
y_circ = O_s[1] + r_o * np.sin(theta_circle)
z_circ = np.zeros_like(theta_circle)

fig.add_trace(go.Scatter3d(
    x=x_circ, y=y_circ, z=z_circ,
    mode="lines",
    line=dict(color="red", width=4, dash="dot"),
    name="rotation circle",
    showlegend=False
))

# Equal axis scaling
all_points = np.column_stack([O_s, O_e, O] + b_axis_endpoints)
max_range = np.ptp(all_points)
center = np.mean(all_points, axis=1)

x_range = [center[0] - max_range/2 - 2.5, center[0] + max_range/2 + 1]
y_range = [center[1] - max_range/2 - 2.5, center[1] + max_range/2 + 1]
z_range = [center[2] - max_range/2 - 2.5, center[2] + max_range/2 + 1]

# Layout
fig.update_layout(
    title="Frames e to b",
    scene=dict(
        xaxis=dict(title="X", range=x_range),
        yaxis=dict(title="Y", range=y_range),
        zaxis=dict(title="Z", range=z_range),
        aspectmode="manual",
        aspectratio=dict(x=1, y=1, z=1)
    ),
    width=950,
    height=850
)

fig.show()


$\mathbf{a}=\mathbf{e}R_3(\alpha)$, $\mathbf{c}\,//\,\mathbf{a}$, $\mathbf{d}=\mathbf{c}R_3(\beta)$, $\mathbf{f}=\mathbf{d}R_1(\phi)$ and $\mathbf{f}\,//\,\mathbf{b}$.

Hence
\begin{align}
\mathbf{b}&=\mathbf{e}R_3(\alpha)R_3(\beta)R_1(\phi)=\mathbf{e}R_3(\alpha+\beta )R_1(\phi)
\end{align}
Thus we have $\mathbf{b}=\mathbf{e}R$ where $R=R_3(\alpha+\beta )R_1(\phi)$. Define $\theta=\alpha+\beta$.

## Angular velocity of the earth frame

The angular velocity of the frame $\mathbf{b}$ with respect to $\mathbf{e}$ is the $\mathbb{R}^3$ version of $\widehat{\Omega}=R^T\dot{R}$.

\begin{align}
\widehat{\Omega}&=R_1^TR_3^T(\dot{R}_3R_1+R_3\dot{R}_1)\\
=&R_1^T(R_3^T\dot{R}_3R_1+\dot{R}_1)\\
=&R_1^T(R_3^T\dot{R}_3)R_1+R_1^T\dot{R}_1\\
\end{align}
which gives
\begin{align}
\Omega=&\dot{\theta}\:R_1^T{e}_3+\dot{\phi}\:{e}_1\\
\end{align}
where
$e_1=[1\:0\:0]^T$ and $e_3=[0\:0\:1]^T$.

In [ ]:
init_vprinting()
t =symbols('t')
t, phi=symbols('t, phi')
theta=dynamicsymbols('theta',real=True)

In [ ]:
R1_phi = Matrix([
    [1, 0, 0],
    [0, cos(phi), -sin(phi)],
    [0, sin(phi),  cos(phi)]
])
R3_theta = Matrix([
    [cos(theta), -sin(theta), 0],
    [sin(theta),  cos(theta), 0],
    [0,           0,          1]
])

R = R3_theta @ R1_phi

In [ ]:
R

In [ ]:
hatOmega=simplify(R.T @ R.diff(t))

In [ ]:
Omega=Matrix([-hatOmega[1,2],hatOmega[0,2],-hatOmega[0,1]])

In [ ]:
Omega

In [ ]:
hatOmegadot=hatOmega.diff(t)

In [ ]:
hatOmegadot

## Newton's equations in the earth frame

The motion variables of of an object in the intertial frame $\mathbf{e}$ will be denoted by lower case letters and the corresponding erth frame variables will be denoted by upper case letters.

Let $o_e=r_oe_1$ and $o=r_ee_2$ where
$e_1=[1\:0\:0]^T$ and $e_2=[0\:1\:0]^T$. Then the motion variables in the inertial frame $\mathbf{e}$ are gien by:

\begin{align}
x&=R_3(\alpha)o_e+Ro+RX\\
\dot{x}&=r_o\dot{\alpha}\,e_2+R\left(\widehat{\Omega}(r_ee_2+X)+\dot{X}\right)\\
\ddot{x}&=r_o\ddot{\alpha}\,e_2+R\left(\widehat{\Omega}^2(r_ee_2+X)+2\widehat{\Omega}\dot{X}+\widehat{\dot{\Omega}}(r_ee_2+X)+\ddot{X}\right)
\end{align}


If $F$ is the forces acting on the particle and represented in the $\mathbf{b}$ frame we have from Newtons equations (assuming that the $\mathbf{b}$ frame is an inertial frame):
\begin{align}
m\ddot{x}&=RF
\end{align}
which gives
\begin{align}
m\left(r_o\ddot{\alpha}\,R^Te_2+\widehat{\Omega}^2(r_ee_2+X)+2\widehat{\Omega}\dot{X}+\widehat{\dot{\Omega}}(r_ee_2+X)+\ddot{X}\right)&=F
\end{align}

For a particle moving under gravity $F=[0\:-mg\:0]^T$

In [ ]:
init_vprinting()
m, r_o, r_e, t, phi=symbols('m, r_o, r_e, t, phi')
x, y, z, alpha, theta=dynamicsymbols('x, y, z, alpha, theta',real=True)
e_2=Matrix([0,1,0])

In [ ]:
X=Matrix([x,z,y])

In [ ]:
Xdot=X.diff(t)
Xddot=X.diff(t,2)

In [ ]:
F=m*(r_o*alpha.diff(t,2)*R @ e_2+hatOmega**2*(r_e*e_2+X)+2*hatOmega*Xdot+hatOmegadot*(r_e*e_2+X)+Xddot)

In [ ]:
F

In [ ]:
F_subs = F.subs({
    phi.diff(t): 0,
    phi.diff(t,2): 0,
    theta.diff(t,2): 0,
    alpha.diff(t,2): 0
})

In [ ]:
F_subs

\begin{align}
m \left(-\dot{\theta}^2 x + 2 \dot{\theta}\dot{y}\sin{\left(\phi \right)} - 2\dot{\theta}\dot{z} \cos{\left(\phi \right)}  + \ddot{x}\right)&=0\\
m \left(\left(y \sin{\left(\phi \right)} \cos{\left(\phi \right)}  - (r_e+z)\cos^{2}{\left(\phi \right)} \right)\dot \theta^{2} + 2 \dot{\theta} \dot{x}\cos{\left(\phi \right)} + \ddot{z}\right)&=-mg\\
m \left(\left(- y \sin^{2}{\left(\phi \right)} + (r_e+z)\sin{\left(\phi \right)} \cos{\left(\phi \right)} \right)\dot{\theta}^{2} - 2 \dot{\theta} \dot{x}\sin{\left(\phi \right)}  + \ddot{y}\right)&=0
\end{align}

## Simulation

The dynamic system form of these equations are then:

\begin{align}
\dot{x}&=v_x,\\
\dot{v}_x&=\dot{\theta}^2 x - 2 \dot{\theta}\dot{y}\sin{\left(\phi \right)} + 2\dot{\theta}\dot{z} \cos{\left(\phi \right)}  \\
\dot{z}&=v_z,\\
\dot{v}_z&=- 2 \dot{\theta} \dot{x}\cos{\left(\phi \right)}-\left(y \sin{\left(\phi \right)} \cos{\left(\phi \right)}  - (r_e+z)\cos^{2}{\left(\phi \right)} \right)\dot \theta^{2}  - g\\
\dot{y}&=v_y,\\
\dot{v}_y&=\left( y \sin^{2}{\left(\phi \right)} - (r_e+z)\sin{\left(\phi \right)} \cos{\left(\phi \right)} \right)\dot{\theta}^{2} + 2 \dot{\theta} \dot{x}\sin{\left(\phi \right)}
\end{align}

In [ ]:
def rotating_earth_system(X, t, parameters):
    """
    X = [x, vx, y, vy, z, vz]
    parameters = [dtheta, phi, r_e, g]

    Returns dX/dt = [dx, dvx, dy, dvy, dz, dvz]
    """
    # Unpack state
    x=X[0]; vx=X[1]; y=X[2]; vy=X[3]; z=X[4]; vz=X[5]

    # Unpack parameters
    dtheta = parameters[0]   # angular velocity (Earth’s rotation rate)
    phi     = parameters[1]  # latitude (radians)
    r_e     = parameters[2]  # Earth’s radius
    g       = parameters[3]  # gravitational acceleration

    # Kinematics
    dx = vx
    dy = vy
    dz = vz

    # Dynamics
    dvx = dtheta**2 * x - 2*dtheta*vy*np.sin(phi) + 2*dtheta*vz*np.cos(phi)
    dvy = (y*np.sin(phi)**2 - (r_e + z)*np.sin(phi)*np.cos(phi))*dtheta**2 + 2*dtheta*vx*np.sin(phi)
    dvz = -2*dtheta*vx*np.cos(phi) - (y*np.sin(phi)*np.cos(phi) - (r_e + z)*np.cos(phi)**2)*dtheta**2 - g

    return [dx, dvx, dy, dvy, dz, dvz]


In [ ]:
import numpy as np

# --- Realistic Earth parameters ---
omega_nom = 7.2921159e-5      # Earth's angular speed [rad/s]
phi_nom   = np.deg2rad(45.0)  # latitude [radians]
r_e_nom   = 6.371e6           # Earth's radius [m]
g_nom     = 9.80665           # gravity [m/s^2]

params = [omega_nom, phi_nom, r_e_nom, g_nom]

# --- Initial state: [x, vx, y, vy, z, vz] ---
# Start 100 m above surface, stationary in the rotating Earth frame
X0 = [0.0, 0.0, 0.0, 0.0, 100.0, 0.0]

# --- Simulation call (25 s fall, 0.05 s step) ---
[t, sol, fig] = mr.simulate_dy_system(
    rotating_earth_system,
    5,          # total simulation time [s]
    0.1,        # time step [s]
    X0,          # initial conditions
    params,      # parameters list
    'Cannonball_Drop_from_Balloon',
    'Time / s',
    'State Variables'
)


In [ ]:
# Extract positions
x_vals = sol[:, 0]
y_vals = sol[:, 2]
z_vals = sol[:, 4]
xx = list(zip(x_vals, y_vals, z_vals))

# Axis limits for a local 200 m cube
axis_range = [(-2, 2), (-20, 120), (-2, 2)]

fig_anim = mr.animate_particle_motion(
    xx,
    axis_range,
    "Cannonball Drop from 100 m Balloon (Rotating Earth Frame)"
)


## Angluar momentum of a particle


The angular momentum of the particle about the origin of the earth fixed frame $\mathbf{b}$:
\begin{align}
\pi&=(RX)\times m\dot{x}\\
&=m\left(RX\right)\times\left(r_o\dot{\alpha}\,e_2+R\left(\widehat{\Omega}(r_ee_2+X)+\dot{X}\right)\right)\\
&=R\left(-m\widehat{X}^2\Omega+mX\times
\left(r_o\dot{\alpha}\,R^Te_2+r_e\widehat{\Omega}e_2+\dot{X}\right)
\right)
\end{align}
where $\mathbb{I}_p\triangleq-m\widehat{X}^2$.

Thius the angular momentum of the about the origin of the earth fixed frame $\mathbf{b}$ expressed in the earth fixed frame $\mathbf{b}$ is

\begin{align}
\Pi&=-m\widehat{X}^2\Omega+mX\times
\left(r_o\dot{\alpha}\,R^Te_2+r_e\widehat{\Omega}e_2+\dot{X}\right)
\end{align}

Noting that $\Omega, \dot{\alpha}, X$ are very small compared to $r_e$ we have
\begin{align}
\Pi&\approx mr_e\,X\times
\widehat{\Omega}e_2
\end{align}